<a href="https://colab.research.google.com/github/ChrisM2309/finance-bot/blob/main/FinanceBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain

In [ ]:
pip install langchain_community openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install openai

Haciendo los import y configurando la api key

In [ ]:
import os
from openai import OpenAI as OpenAI

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.llms import OpenAI as OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

import json

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

client = OpenAI(
    api_key = OPENAI_API_KEY
)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

<ipython-input-5-7c5c98998788>:17: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(


In [ ]:
#ChatBot Type
chat = ChatOpenAI(model="gpt-4o-mini")

<ipython-input-6-44457f2654b1>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model="gpt-4o-mini")


# CREAR TOOLS


In [ ]:
# CHAINS
# Crear una cadena (Chain) para registrar transacciones
prompt_registrar = PromptTemplate(
    input_variables=["input"],
    template=""""
    Del input recibido, analiza con precaucion y extrae los parametros necesarios para hacer el registro:
    - Tipo (Ingreso o Gasto)
    - Monto (número)
    - Categoría (texto)
    - Descripción (texto, opcional, usa '' si no está presente)

    Texto: "{input}"

    Devuelve la respuesta en formato "(tipo) registrado con (monto) en (categoria) con (descripcion).".
    """
)

chain_registrar = LLMChain(llm=chat, prompt=prompt_registrar)

tool_registrar = Tool(
    name="registrar_transaccion",
    func=lambda x: chain_registrar.run(input=x),
    description="Sirve para registrar un ingreso o gasto con tipo (Ingreso/Gasto), monto (número), categoria (texto) y descripcion (texto) (opcional).")


# CALCULO DE FLUJO DE CAJA
prompt_flujo_caja = PromptTemplate(
    input_variables=["input"],
    template="""
    Analiza el siguiente texto y extrae los parámetros para calcular el flujo de caja:
    - Ingresos (número)
    - Gastos (número)

    Texto: "{input}"

    Calcula el saldo (ingresos - gastos) y devuelve el resultado en formato:
    "🔹 Ingresos: $(ingresos) 🔹 Gastos: $(gastos)🔹 Saldo: $(saldo)"
    """
)
chain_flujo_caja = LLMChain(llm=chat, prompt=prompt_flujo_caja)

tool_flujo_caja = Tool(
    name="calcular_flujo_caja",
    func=lambda x: chain_flujo_caja.run(input=x),
    description="Calcula el flujo de caja a partir de ingresos y gastos."
)


# CONFIGURAR PRESUPUESTO
prompt_presupuesto = PromptTemplate(
    input_variables=["input"],
    template="""
    Analiza el siguiente texto y extrae los parámetros para configurar un presupuesto:
    - Categoría (texto)
    - Monto (número)

    Texto: "{input}"

    Devuelve el resultado en formato: "✅ Presupuesto de ${monto} para {categoria} configurado."
    """
)

chain_presupuesto = LLMChain(llm=chat, prompt=prompt_presupuesto)

tool_presupuesto = Tool(
    name="configurar_presupuesto",
    func= lambda x: chain_presupuesto.run(input=x),
    description="Configura un presupuesto para una categoría."
)

# RESPONDER PREGUNTAS FINANCEIRAS
prompt_preguntas = PromptTemplate(
    input_variables=["input"],
    template="""
    Responde la siguiente pregunta financiera de manera clara y sencilla:
    "{input}"
    Usa toda la informacion disponible para responder la pregunta. Si es necesarios, debes consultar la memoria para obtener datos relevantes.
    """
)
chain_preguntas = LLMChain(llm=chat, prompt=prompt_preguntas)

tool_preguntas = Tool(
    name="responder_pregunta",
    func=lambda x: chain_preguntas.run(input=x),
    description="Responde preguntas financieras generales o relacionadas con la memoria"
)

In [ ]:
# Añadir la herramienta procesar_multiples tools e indicaciones
# Prompt para descomponer el mensaje
prompt_descomponer = PromptTemplate(
    input_variables=["input"],
    template="""
    Analiza el siguiente texto y descompónlo en una lista de tareas a ejecutar. Para cada tarea, identifica:
    - Herramienta a usar: "registrar_transaccion", "calcular_flujo_caja", "configurar_presupuesto",  "responder_pregunta", etc...
    - Input específico para esa herramienta (como texto o parámetros)

    Texto: {input}

    Para el output:
    Devuelve el resultado como una lista de objetos JSON, donde cada objeto tiene:
    "tool": nombre de la herramienta
    "input": texto que contiene el input de la herramienta, con todos los parametros necesarios para ejecutarla.

    Ejemplo:
    Input: "Registra un ingreso de $100 en ventas y calcula el flujo de caja con ingresos de $500 y gastos de $300."
    Output: [
        {{"tool": "registrar_transaccion", "input": "Registra un ingreso de $100 en ventas."}},
        {{tool": "calcular_flujo_caja", "input": "Calcula el flujo de caja con ingresos de $500 y gastos de $300."}}
    ]
    """
)
#chain para descomponer
chain_descomponer = LLMChain(llm=chat, prompt=prompt_descomponer)

#Funcion para la implementacion de tool
def procesar_multiples_tool(input_text):
    tareas_json = chain_descomponer.run(input=input_text)
    try:
        tareas = json.loads(tareas_json)
    except json.JSONDecodeError as e:
        return f"Error al procesar las tareas: {str(e)}"

    herramientas = {
        "registrar_transaccion": lambda x: chain_registrar.run(input=x),
        "calcular_flujo_caja": lambda x: chain_flujo_caja.run(input=x),
        "configurar_presupuesto": lambda x: chain_presupuesto.run(input=x),
        "responder_pregunta": lambda x: chain_preguntas.run(input=x),
        "manejar_deudas": lambda x: chain_deudas.run(input=x)
        # Agrega más herramientas según sea necesario
    }
    resultados = []
    for tarea in tareas:
        tool_name = tarea["tool"]
        tool_input = tarea["input"]
        resultado = herramientas[tool_name](tool_input)
        resultados.append(resultado)
    return "\n".join(resultados)
#instancia de tool
tool_multiples = Tool(
    name="procesar_multiples",
    func=procesar_multiples_tool,
    description="Procesa un mensaje con múltiples instrucciones y ejecuta las herramientas en orden."
)

In [ ]:
#Chain para manejo de deudas

#Prompt de deudas
prompt_deudas = PromptTemplate(
    input_variables=["input"],
    template="""
    Del siguiente texto, extrae los parametros para manejar un deuda:
    - Acccion (agregar, pagar, consultar) o similares
    - Monto (número)
    - Plazo (número)
    - Acreedor (texto)
    - Interés (número, opcional)
    - Fecha de vencimiento (fecha)
    - Descripción (texto, opcional, usa '' si no está presente)

    Texto: "{input}"

    Devuelve la respuesta en formato:
    Segun la accion que se quiera realizar, devolver una respuesta diferente:
    - Agregar: "Deuda de $(monto) a (acreedor) con vencimiento (fecha_vencimiento) agregada" (Agregar 'interes' y otros parametros si existen)
    - Pagar: "Deuda de $(monto) a (acreedor) con vencimiento (fecha_vencimiento) pagada" (Agregar 'interes' y otros parametros si existen)
    - Consultar: "Lista de deudas: (lista_deudas)"
    """
)
def manejar_deudas_tool(input_text):
    return chain_deudas.run(input=input_text)\

#chain de deudas
chain_deudas = LLMChain(llm=chat, prompt=prompt_deudas)
tool_deudas = Tool(
    name = "manejar_deudas",
    func = manejar_deudas_tool,
    description = "Sirve para manejar deudas. Puede agregar, pagar o consultar deudas."
)


In [ ]:
#Balance general
prompt_balance	= PromptTemplate(
    input_variables=["input"],
    template="""
    PARA ESTA TAREA ES ESENCIAL QUE CONSULTES LA MEMORIA DE BUFFER EXISTENTE Y/O EL TEXTO INGRESADO PARA EXTRAER LOS PARAMETROS NECESARIOS PARA GENERAR UN BALANCE GENERAL:
    Comsulta la memoria existente y/o el texto ingresado para extraer los parámetros para generar un balance general:
    usa toda la informacion que tengas disponible para completar lo siguiente:
    - Suma de todos los ingresos registrados : CONSULTA EN LA MEMORIA TODOS LOS INGRESOS QUE SE HAN REGISTRADO Y SUMALOS
    - Suma de todos los gastos registrados: CONSULTA EN LA MEMORIA TODOS LOS GASTOS QUE SE HAN REGISTRADO Y SUMALOS
    - Calcular el balance (ingresos - gastos) : RESTA LOS VALORES QUE OBTUVISTE

    Texto: "{input}"

    Devuelve el resultado en formato:
    "Balance general: Ingresos $(ingresos) - $(gastos) = $(balance)"
    """
)

chain_balance = LLMChain(llm = chat, prompt = prompt_balance)

tool_balance = Tool(
    name = "balance_general",
    func = lambda x: chain_balance.run(input = x),
    description = "Calcula el balance general de ingresos y gastos."
)

# CONFIGURAR AL AGENTE

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")

# Inicializar el agente
# Lista de herramientas
tools = [
    tool_registrar,
    tool_flujo_caja,
    tool_presupuesto,
    tool_preguntas,
    tool_multiples,
    tool_deudas,
    tool_balance
  ]

# Inicializar el agente
agente = initialize_agent(
    tools = tools,
    llm = chat,
    agent="conversational-react-description",
    verbose=True,
    max_iterations = 10,
    memory = memory
    )

TESTEO DEL AGENTE

In [ ]:
print(agente.run("Quiero registrar un pago de $30 en electricidad, uno de $40 en internet. Luego, tengo que registrar un ingreso de $100 en ventas. Luego, calcula el flujo de caja con los valores anteriores y por ultimo, asigna una deuda de $40 para el 20 de marzo de 2025"))



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: procesar_multiples
Action Input: [
    {"action": "registrar_transaccion", "type": "Gasto", "amount": 30, "category": "Electricidad", "description": "Pago de electricidad"},
    {"action": "registrar_transaccion", "type": "Gasto", "amount": 40, "category": "Internet", "description": "Pago de internet"},
    {"action": "registrar_transaccion", "type": "Ingreso", "amount": 100, "category": "Ventas", "description": "Ingreso por ventas"},
    {"action": "calcular_flujo_caja", "amounts": [100, -30, -40]},
    {"action": "manejar_deudas", "type": "agregar", "amount": 40, "due_date": "2024-03-20"}
]
```

Observation: Error al procesar las tareas: Expecting value: line 1 column 1 (char 0)
Thought:Do I need to use a tool? Yes
Action: procesar_multiples
Action Input: [
    {"action": "registrar_transaccion", "type": "Gasto", "amount": 30, "category": "Electricidad", "description": "Pago de electricidad"},
 

In [ ]:
#print(agente.run("Registra un gasto de $150 en insumos y un ingreso de $200 en ventas."))
print(agente.run("¿Qué es el flujo de caja?"))
print(agente.run("Calcula el flujo de caja con ingresos de $500 y gastos de $300."))
print(agente.run("Registra un gasto de $200 en publiicidad."))
print(agente.run("Cuanto he gastado en total?"))
print(agente.run("¿Qué es el flujo de caja?"))




> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
AI: El flujo de caja es un término financiero que se refiere al movimiento de dinero dentro y fuera de una empresa o entidad en un período determinado. Es un indicador clave de la salud financiera de una organización, ya que muestra cómo se generan y utilizan los fondos. 

Un flujo de caja positivo significa que una empresa está generando más dinero del que gasta, lo que es generalmente una señal de buena salud financiera. Por otro lado, un flujo de caja negativo puede indicar problemas financieros, ya que significa que los gastos superan a los ingresos.

El flujo de caja se puede dividir en tres categorías:
1. **Flujo de caja de operaciones**: Dinero generado o utilizado en las actividades principales de la empresa, como ventas y gastos operativos.
2. **Flujo de caja de inversión**: Dinero gastado o recibido a través de inversiones en activos a largo plazo, como la compra o venta de propiedades o equipos.

In [ ]:
print(agente.run("Responde como pregunta financiera, dame un resumen de todas las operaciones que he realizado"))



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: responder_pregunta
Action Input: Dame un resumen de todas las operaciones realizadas.
Observation: Para poder ofrecerte un resumen de todas las operaciones realizadas, necesitaría información específica sobre las transacciones o actividades financieras en cuestión. Sin embargo, puedo explicarte cómo estructurar un resumen de operaciones financieras, que generalmente incluye:

1. **Fecha de la operación**: Indica cuándo se realizó cada transacción.
2. **Descripción de la operación**: Detalla qué tipo de operación fue, por ejemplo, compra de activos, venta de productos, pago de deudas, etc.
3. **Monto de la operación**: Especifica el valor monetario involucrado en cada transacción.
4. **Parte involucrada**: Menciona quién realizó la operación y con quién se realizó (proveedores, clientes, etc.).
5. **Resultado de la operación**: Indica el efecto de la operación en la situación financiera (ganancia, 

In [ ]:
print(agente.run("Calcula el balance general de ingresos y gastos."))



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: balance_general
Action Input: {}
Observation: Parece que no tengo acceso a información previa o a una memoria de datos para consultar los ingresos y gastos registrados. Sin embargo, si me proporcionas los datos de ingresos y gastos, puedo ayudarte a calcular el balance general. Por favor, indícame los valores para proceder.
Thought:Do I need to use a tool? No  
AI: Parece que no tengo acceso a información previa o a una memoria de datos para consultar los ingresos y gastos registrados. Sin embargo, si me proporcionas los datos de ingresos y gastos, puedo ayudarte a calcular el balance general. Por favor, indícame los valores para proceder.

> Finished chain.
Parece que no tengo acceso a información previa o a una memoria de datos para consultar los ingresos y gastos registrados. Sin embargo, si me proporcionas los datos de ingresos y gastos, puedo ayudarte a calcular el balance general. Por favor,